In [3]:
# %% [code]
import duckdb  # pip install duckdb (必要に応じて)
import pandas as pd
import numpy as np
import os
from google.colab import drive

# ----------------------------------------------------------
# 1. Google Driveマウント & パス設定
# ----------------------------------------------------------
drive.mount('/content/drive/')

# DuckDBのDBファイルを保存するパス
db_path = "/content/drive/MyDrive/Colab Notebooks/standard/rossmann-store-sales/data/dwh.duckdb"

# クリーニング済みデータ(csv)
cleaned_data_path = "/content/drive/MyDrive/Colab Notebooks/standard/rossmann-store-sales/data/cleaned_data.csv"

# ----------------------------------------------------------
# 2. DuckDBへ接続 (なければ新規作成される)
# ----------------------------------------------------------
con = duckdb.connect(db_path)
print(f"[INFO] Connected to DuckDB at: {db_path}")

# ----------------------------------------------------------
# 3. cleaned_data.csv をDuckDBテーブルとしてロード
#    ※read_csv_autoは内部的に推論して取り込みます。
#    Date列を自動で日付型にしたい場合は、dateformatなどの設定を考慮
# ----------------------------------------------------------
con.execute(f"""
CREATE OR REPLACE TABLE cleaned_data AS
SELECT *
FROM read_csv_auto('{cleaned_data_path}', header=True, dateformat='%Y-%m-%d');
""")
print("[INFO] cleaned_data テーブルを作成しました。")

# 行数確認
row_count = con.execute("SELECT COUNT(*) FROM cleaned_data").fetchone()[0]
print("cleaned_data row count:", row_count)

# ----------------------------------------------------------
# 4. dim_store (店舗ディメンション) 作成
#    - 店舗に関する情報を取り出し、重複を排除して1テーブル化
#    - store_key (連番) を付与し、Store(自然キー)と切り離すのがポイント
# ----------------------------------------------------------
con.execute("""
CREATE OR REPLACE TABLE dim_store AS
    SELECT
        row_number() OVER (ORDER BY Store) AS store_key,
        Store,
        StoreType,
        Assortment,
        CompetitionDistance,
        CompetitionOpenSinceMonth,
        CompetitionOpenSinceYear,
        Promo2,
        Promo2SinceWeek,
        Promo2SinceYear,
        PromoInterval
    FROM (
        SELECT DISTINCT
            Store,
            StoreType,
            Assortment,
            CompetitionDistance,
            CompetitionOpenSinceMonth,
            CompetitionOpenSinceYear,
            Promo2,
            Promo2SinceWeek,
            Promo2SinceYear,
            PromoInterval
        FROM cleaned_data
    )
    ORDER BY Store;
""")
print("[INFO] dim_store テーブルを作成しました。")

# ----------------------------------------------------------
# 5. dim_date (日付ディメンション) 作成
#    - データ内に出現する日付を一意に抜き出し、date_key(YYYYMMDDなど)を付与
# ----------------------------------------------------------
con.execute("""
CREATE OR REPLACE TABLE dim_date AS
WITH distinct_dates AS (
    SELECT DISTINCT
        Date,
        Year,
        Month,
        DayOfWeek,
        WeekOfYear
    FROM cleaned_data
)
SELECT
    CAST(strftime('%Y%m%d', Date) AS INTEGER) AS date_key,
    Date,
    Year,
    Month,
    DayOfWeek,
    WeekOfYear
FROM distinct_dates
ORDER BY Date;
""")
print("[INFO] dim_date テーブルを作成しました。")

# ----------------------------------------------------------
# 6. fact_sales (売上ファクトテーブル) 作成
#    - 主に売上関連のメトリクスを持つテーブル
#    - 店舗を store_key, 日付を date_key で参照
# ----------------------------------------------------------
con.execute("""
CREATE OR REPLACE TABLE fact_sales AS
WITH joined AS (
    SELECT
        c.*,
        d.store_key,
        dt.date_key
    FROM cleaned_data c
    LEFT JOIN dim_store d
        ON c.Store = d.Store
    LEFT JOIN dim_date dt
        ON c.Date = dt.Date
)
SELECT
    store_key,
    date_key,
    Sales,
    Customers,
    Open,
    Promo,
    SchoolHoliday,
    StateHoliday
FROM joined;
""")
print("[INFO] fact_sales テーブルを作成しました。")

# サンプル確認
fact_count = con.execute("SELECT COUNT(*) FROM fact_sales;").fetchone()[0]
print("fact_sales row count:", fact_count)

# ----------------------------------------------------------
# 7. 一枚のワイドテーブル data_mart_wide (オプション)
#    - 「スタースキーマは複雑…」という場合や小規模分析用に便利
# ----------------------------------------------------------
con.execute("""
CREATE OR REPLACE TABLE data_mart_wide AS
SELECT
    c.*,  -- cleaned_dataの全列
    d.store_key, -- 追加で参照
    dt.date_key  -- 追加で参照
FROM cleaned_data c
LEFT JOIN dim_store d
    ON c.Store = d.Store
LEFT JOIN dim_date dt
    ON c.Date = dt.Date;
""")
wide_count = con.execute("SELECT COUNT(*) FROM data_mart_wide;").fetchone()[0]
print("[INFO] data_mart_wide テーブルを作成しました。")
print("data_mart_wide row count:", wide_count)

# ----------------------------------------------------------
# 8. 確認 & クローズ
# ----------------------------------------------------------
# 例: dim_storeの先頭5件を表示
store_sample = con.execute("SELECT * FROM dim_store LIMIT 5;").fetchdf()
print("\n[SAMPLE] dim_store:")
print(store_sample)

# 例: fact_salesの先頭5件
fact_sample = con.execute("""
SELECT fact_sales.*, dim_date.Date AS actual_date
FROM fact_sales
LEFT JOIN dim_date ON fact_sales.date_key = dim_date.date_key
LIMIT 5;
""").fetchdf()
print("\n[SAMPLE] fact_sales joined with dim_date:")
print(fact_sample)

# DuckDB接続を閉じる
con.close()
print("\n[INFO] DWH Transformation in DuckDB completed & connection closed.")


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[INFO] Connected to DuckDB at: /content/drive/MyDrive/Colab Notebooks/standard/rossmann-store-sales/data/dwh.duckdb


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[INFO] cleaned_data テーブルを作成しました。
cleaned_data row count: 560478
[INFO] dim_store テーブルを作成しました。
[INFO] dim_date テーブルを作成しました。
[INFO] fact_sales テーブルを作成しました。
fact_sales row count: 560478
[INFO] data_mart_wide テーブルを作成しました。
data_mart_wide row count: 560478

[SAMPLE] dim_store:
   store_key  Store StoreType Assortment  CompetitionDistance  \
0          1      1         c          a               1270.0   
1          2      2         a          a                570.0   
2          3      3         a          a              14130.0   
3          4      4         c          c                620.0   
4          5      5         a          a              29910.0   

   CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
0                          9                      2008       0   
1                         11                      2007       1   
2                         12                      2006       1   
3                          9                      2009       0   
4      

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
